# tsam - Segmentation
Example usage of the time series aggregation module (tsam)
Date: 31.10.2019

Author: Maximilian Hoffmann

Import pandas and the relevant time series aggregation class

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd

import tsam
from tsam import ClusterConfig, SegmentConfig

### Input data 

Read in time series from testdata.csv with pandas

In [ ]:
raw = pd.read_csv("testdata.csv", index_col=0)

Create a plot function for the temperature for a visual comparison of the time series

In [ ]:
# Use tsam's built-in plotting with plotly
# tsam.plot_heatmap(data, column, period_hours) creates interactive heatmaps

### Hierarchical aggregation with medoid representation and 10 typical days with 24 hourly segments

Initialize an aggregation class object with hierarchical as method for eight typical days

In [ ]:
result = tsam.aggregate(
    raw,
    n_periods=10,
    period_hours=24,
    cluster=ClusterConfig(method="hierarchical"),
)

Create the typical periods

In [ ]:
typPeriods = result.typical_periods
typPeriods

Predict original data

In [ ]:
predictedPeriods = result.reconstruct()
predictedPeriods

Get accuracy indicators

In [ ]:
result.accuracy

### Hierarchical aggregation with medoid representation and 20 typical days with 12 irregular segments

In [ ]:
resultSeg = tsam.aggregate(
    raw,
    n_periods=20,
    period_hours=24,
    cluster=ClusterConfig(method="hierarchical"),
    segments=SegmentConfig(n_segments=12),
)

Create the typical periods

In [ ]:
typPeriodsSeg = resultSeg.typical_periods
typPeriodsSeg

Predict original data

In [ ]:
predictedPeriodsSeg = resultSeg.reconstruct()
predictedPeriodsSeg

Get accuracy indicators

In [ ]:
resultSeg.accuracy

### Comparison of the aggregations 
It was shown for the temperature, but both times all four time series have been aggregated. Therefore, we compare here also the duration curves  of the electrical load for the original time series, the aggregation with k-mean, and the hierarchical aggregation including peak periods.

In [ ]:
# Compare duration curves using built-in function
tsam.plot.compare_results(
    {
        "Original": raw,
        "10 with 24 hours": predictedPeriods,
        "20 with 12 Seg": predictedPeriodsSeg,
    },
    column="Load",
    plot_type="duration_curve",
)

In [ ]:
param = "GHI"

In [ ]:
# Original data heatmap
tsam.plot_heatmap(raw, column=param, period_hours=24, title=f"Original {param}")

In [ ]:
# 10 periods with 24 hours heatmap
tsam.plot_heatmap(
    predictedPeriods, column=param, period_hours=24, title=f"10 with 24 hours - {param}"
)

In [ ]:
# 20 periods with 12 segments heatmap
tsam.plot_heatmap(
    predictedPeriodsSeg,
    column=param,
    period_hours=24,
    title=f"20 with 12 Seg - {param}",
)

In [ ]:
# Time slice comparison using built-in function
tsam.plot.compare_results(
    {
        "Original": raw,
        "10 with 24 hours": predictedPeriods,
        "20 with 12 seg": predictedPeriodsSeg,
    },
    column="Load",
    plot_type="time_slice",
    start="20100210",
    end="20100218",
)

### Validation

Check that the means of the original time series and the predicted ones are the same.

In [ ]:
raw.mean()

In [ ]:
predictedPeriods.mean()

In [ ]:
predictedPeriodsSeg.mean()

Check that a segmented period has the same column-wise means as a non-segmented period for if the periods are the same.

In [ ]:
# Mean of first period with non-segmented aggregation
result.typical_periods.loc[0, :].mean()

In [ ]:
# Segmented aggregation with same number of periods for comparison
resultSegTest = tsam.aggregate(
    raw,
    n_periods=10,
    period_hours=24,
    cluster=ClusterConfig(method="hierarchical"),
    segments=SegmentConfig(n_segments=12),
)

In [ ]:
# Get segment durations
segmentDurations = resultSegTest.segment_durations
print("Segment durations:", segmentDurations)

In [ ]:
# Weighted mean of first period (should match non-segmented period mean)
period_0 = resultSegTest.typical_periods.loc[0, :].reset_index(0, drop=True)
seg_durations = list(segmentDurations.values())
weighted_mean = period_0.mul(seg_durations, axis=0).sum() / sum(seg_durations)
weighted_mean

Print out the (segmented) typical periods.

In [ ]:
# Display segmented typical periods
resultSeg.typical_periods

In [ ]:
# Display non-segmented typical periods
result.typical_periods